# CPAS Applications Query

In [15]:
# Connecting to CPAS with cx_Oracle
# cx_Oracle is a Python extension module that enables access to Oracle Database
import cx_Oracle
import sqlalchemy
import os
import urllib
import pandas as pd

In [16]:
USERNAME = os.environ.get('CPAS_USERNAME')
PASSWORD = os.environ.get('CPAS_PASSWORD')
HOSTNAME = '10.32.196.224'
PORT = '1332'
SERVICE_NAME = 'visispd1.lacity.org'

dsn = cx_Oracle.makedsn(HOSTNAME, PORT, service_name=SERVICE_NAME)
connection = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn)

In [17]:
import sqlalchemy

engine = sqlalchemy.create_engine(
f"oracle+cx_oracle://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/"
f"?service_name={SERVICE_NAME}"
)


In [18]:
USER = "GRNT44"
engine.execute(
    f"SELECT table_name FROM all_tables WHERE owner='{USER}'"
).fetchall()

[('GRNT_PRR_YR_CDBG',),
 ('GRNT_SL_DEPT_CNTCT',),
 ('GRNT_SRC_FND',),
 ('GRNT_STAT',),
 ('SERV_AREA_CENSUS_TRACT',),
 ('GRNT_PEP_GOAL_OTCM',),
 ('GRNT_L_ACCMPLSH_CATG',),
 ('GRNT_PEP_ACCMPLSH',),
 ('GRNT_GPR_ATTCHMNT',),
 ('GRNT_PRJCT_FND_AMND',),
 ('GRNT_L_RACE',),
 ('GRNT_GPR_RACE',),
 ('GRNT_GPR_CENSUS_TRACT',),
 ('GRNT_CENSUS_TRACT',),
 ('GRNT_GPR_IDIS_ACTV_TRKNG',),
 ('GRNT_CPTL_PLAN',),
 ('GRNT_CPTL_PLAN_DTL',),
 ('GRNT_CPTL_TYP',),
 ('GRNT_FORM',),
 ('GRNT_GOAL_OTCM',),
 ('GRNT_GPR_DUNS',),
 ('GRNT_GPR_INCM_LVL',),
 ('GRNT_GPR_JOB_CATG',),
 ('GRNT_HIST',),
 ('GRNT_HSTRCL_RVW',),
 ('GRNT_HUD_CD',),
 ('GRNT_HUD_OBJCTIVE',),
 ('GRNT_LOCTN_TYP',),
 ('GRNT_L_ACCMPLSH',),
 ('GRNT_L_APLC_ELIG',),
 ('GRNT_L_APLC_READINESS',),
 ('GRNT_L_ATTCHMNT',),
 ('GRNT_L_CBDO_COLOR',),
 ('GRNT_L_CBDO_DOC',),
 ('GRNT_L_CBDO_STTS',),
 ('GRNT_L_CNCL_DIST',),
 ('GRNT_L_CONPLAN_CATG',),
 ('GRNT_L_DAVIS_BACON',),
 ('GRNT_L_ENV_CEQA_STTS',),
 ('GRNT_L_EXPNS',),
 ('GRNT_L_FND',),
 ('GRNT_L_INCM_LVL',),
 ('G

In [19]:
df = pd.read_sql('SELECT * FROM "GRNT44"."GRNT_STAT"', engine)
df.head(10)

,grnt_stat_id,grnt_l_stat,grnt_aplctn_id
0,473917,197,473909
1,474231,277,474223
2,474303,277,474289
3,474327,277,474319
4,474429,277,474421
5,474465,277,474457
6,474621,277,474611
7,474713,277,474703
8,474923,277,474909
9,474961,277,474947


In [20]:
import pandas
USER = "GRNT44"
engine.execute(f"SELECT table_name FROM all_tables WHERE owner='{USER}'").fetchall()

df = pd.read_sql(
    """SELECT * FROM
            (SELECT
            apl.grnt_aplctn_id,
            apl.prjct_ttl,
            apl.fnd_yr,
            (SELECT
                    field_value
                FROM
                    grnt44.grnt_l_stat,
                    grnt44.grnt_stat
                WHERE
                    grnt_stat.grnt_l_stat = grnt_l_stat.field_id
                    AND apl.grnt_aplctn_id = grnt_stat.grnt_aplctn_id
             ) AS aplctn_status,

            (SELECT
                    field_value
                FROM
                    grnt.grnt_l_dept,
                    grnt44.grnt_cntct
                WHERE
                    grnt_cntct.grnt_l_dept = grnt_l_dept.field_id
                    AND cty_dept_yn = 1
                    AND apl.grnt_city_dept_cntct_id = grnt_cntct_id
            ) AS dept_nm
        FROM
            grnt44.grnt_aplctn apl
        WHERE
            1 = 1
    )
WHERE
    1 = 1
    AND fnd_yr = 44
ORDER BY
    prjct_ttl,
    dept_nm,
    aplctn_status """, engine)

df.head(10)

,grnt_aplctn_id,prjct_ttl,fnd_yr,aplctn_status,dept_nm
0,475815,5800 FIGUEROA POCKET PARK,44,SUBMITTED,"RECREATION AND PARKS, DEPARTMENT OF"
1,479173,AFFORDABLE HOUSING MANAGED PIPELINE AND PROGRA...,44,SUBMITTED,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT
2,474223,AGING DEPARTMENT,44,SUBMITTED,"AGING, DEPARTMENT OF"
3,474319,AGING SERVICES DELIVERY SYSTEM,44,SUBMITTED,"AGING, DEPARTMENT OF"
4,474289,ALGIN SUTTON POOL,44,SUBMITTED,"RECREATION AND PARKS, DEPARTMENT OF"
5,478745,BOYLE HEIGHTS TECHNOLOGY YOUTHSOURCE CENTER RO...,44,SUBMITTED,ECONOMIC AND WORKFORCE DEVELOPMENT DEPARTMENT
6,478853,BRADLEY MILKEN YOUTH AND FAMILY CENTER - S.E.L...,44,SUBMITTED,ECONOMIC AND WORKFORCE DEVELOPMENT DEPARTMENT
7,477955,BROWNFIELDS REVITALIZATION COMMUNITY-WIDE REVI...,44,SUBMITTED,"PUBLIC WORKS: SANITATION, BUREAU OF"
8,483915,BUILDING IMPROVEMENT FUND,44,SUBMITTED,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT
9,481579,CERTIFIED ACCESS SPECIALISTS (CASP),44,SUBMITTED,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT


In [25]:
import pandas
USER = "GRNT38"
engine.execute(f"SELECT table_name FROM all_tables WHERE owner='{USER}'").fetchall()

df = pd.read_sql(
    """SELECT * FROM
            (SELECT
            apl.grnt_aplctn_id,
            apl.prjct_ttl,
            apl.fnd_yr,
            (SELECT
                    field_value
                FROM
                    grnt38.grnt_l_stat,
                    grnt38.grnt_stat
                WHERE
                    grnt_stat.grnt_l_stat = grnt_l_stat.field_id
                    AND apl.grnt_aplctn_id = grnt_stat.grnt_aplctn_id
             ) AS aplctn_status,

            (SELECT
                    field_value
                FROM
                    grnt.grnt_l_dept,
                    grnt39.grnt_cntct
                WHERE
                    grnt_cntct.grnt_l_dept = grnt_l_dept.field_id
                    AND cty_dept_yn = 1
                    AND apl.grnt_city_dept_cntct_id = grnt_cntct_id
            ) AS dept_nm
        FROM
            grnt38.grnt_aplctn apl
        WHERE
            1 = 1
    )
WHERE
    1 = 1
    AND fnd_yr = 38
ORDER BY
    prjct_ttl,
    dept_nm,
    aplctn_status """, engine)

df.head(20)

,grnt_aplctn_id,prjct_ttl,fnd_yr,aplctn_status,dept_nm
0,343875,13116 WEST KAGEL CANYON POCKET PARK,38,SUBMITTED,None
1,344307,5500-5520 CONCORD AVENUE POCKET PARK,38,SUBMITTED,None
2,344327,8620 WALL STREET POCKET PARK,38,SUBMITTED,None
3,341213,AFFORDABLE HOUSING TRUST FUND PROGRAM DELIVERY,38,SUBMITTED,None
4,346011,AGING DEPARTMENT,38,SUBMITTED,None
5,346625,AGING SERVICES DELIVERY SYSTEM (CBDO),38,SUBMITTED,None
6,346219,AGING SERVICES DELIVERY SYSTEM (PS),38,SUBMITTED,None
7,340749,AIDS POLICY DEVELOPMENT,38,SUBMITTED,None
8,340739,AIDS PREVENTION,38,SUBMITTED,None
9,344633,AVALON PARK PLAZA,38,SUBMITTED,None
